#### FYI: Abandoned experiments, learnings etc can be found here:
[pre_processing_and_wrangling - experiments and leaning](pre_processing_and_wrangling%20-%20experiments%20and%20leaning.ipynb)

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder. \
    appName("pyspark-1"). \
    getOrCreate()

In [3]:
df = spark.read.csv("/dataset/nyc-jobs.csv", header=True, inferSchema=True)

In [4]:
featues = ["Job ID",
           "Agency", 
           "Job Category",
           "Salary Range From",
           "Salary Range To",
           "Salary Frequency",
           "Minimum Qual Requirements",
           "Posting Date"]

In [5]:
df_required_feaures = df.select(featues)
df = None

In [6]:
df_required_feaures.printSchema()

root
 |-- Job ID: integer (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Job Category: string (nullable = true)
 |-- Salary Range From: double (nullable = true)
 |-- Salary Range To: double (nullable = true)
 |-- Salary Frequency: string (nullable = true)
 |-- Minimum Qual Requirements: string (nullable = true)
 |-- Posting Date: string (nullable = true)



In [7]:
df_required_feaures.limit(5).toPandas().transpose()

,0,1,2,3,4
Job ID,87990,97899,132292,132292,133921
Agency,DEPARTMENT OF BUSINESS SERV.,DEPARTMENT OF BUSINESS SERV.,NYC HOUSING AUTHORITY,NYC HOUSING AUTHORITY,NYC HOUSING AUTHORITY
Job Category,None,None,Maintenance & Operations,Maintenance & Operations,Maintenance & Operations
Salary Range From,42405,60740,51907.7,51907.7,35
Salary Range To,65485,162014,54580.3,54580.3,35
Salary Frequency,Annual,Annual,Annual,Annual,Hourly
Minimum Qual Requirements,"""1.\tA baccalaureate degree from an accredited...","""1. A baccalaureate degree from an accredited ...","""1. Three years of full-time satisfactory expe...","""1. Three years of full-time satisfactory expe...",1. Five years of full-time satisfactory experi...
Posting Date,New York City residency is generally required ...,2012-01-26T00:00:00.000,None,None,2014-01-09T00:00:00.000


### Feature Engineering

- Salary 

    - Required: a function to derive a salary based on "Salary Range From", "Salary Range To" and "Salary Frequency"
        The resultant salary must be in the same units across each "Salary Frequency" so that a comparison can be made


- Minimum Qual Requirements 

    - Required: a function to extrapolate any qualification specification.  I may end up having a simply flag:  required_degree: boolean
   
   
- Posting Date - only about half of the records have a valid data in this field
                

## Salary

Required: derive a salary based on "Salary Range From", "Salary Range To" and "Salary Frequency" The resultant salary must be in the same units across each "Salary Frequency" so that a comparison can be made

In [77]:
from pyspark.sql.functions import col,regexp_extract

when "Salary Range From" is not null and not 0, then calculate the midrange salary, i.e. sal = (min + (max - min)/2)
 otherwise use "Salary Range To"

In [9]:
import sys
sys.path.insert(0,'../utils')

In [10]:
import importlib

#### PLEASE NOTE
get_HourlySalaryUDF was tested here [get_HourlySalaryUDF](test%20UDF%20functions.ipynb#test_get_HourlySalaryUDF)

In [11]:
import udf_functions as UDF

In [12]:
help(UDF.get_HourlySalaryUDF)

Help on function get_HourlySalaryUDF in module udf_functions:

get_HourlySalaryUDF(SalaryRangeFrom, SalaryRangeTo, SalaryFrequency) -> float
    Description:
    
    Santity checks of accidental From To swapping:
        if SalaryRangeFrom > SalaryRangeTo
            then swap the values.
    
    Calculate SalaryMixedFreq as follows:
        if "Salary Range From" is not null and not 0, then calculate 
            midrange salary, i.e. sal = (min + (max - min)/2)
        else 
            use "Salary Range To" as the salary
        If SalaryRangeTo is not defined or 0 return None
    
    Calculate HourlySalary as follows:
    
       SalaryMixedFreq will be the yearly salary, Daily salary or 
       hourly salary depending on "Salary Frequency"
       Calculate HourlySalary based on "Salary Frequency":
       1 - "Hourly": set HourlySalary = SalaryMixedFreq
       2 - "Daily":  set HourlySalary = SalaryMixedFreq / 8 rounded to 2 dec places
       3 - "Annual": set HourlySalary = Sal

In [13]:
df_salary = df_required_feaures.withColumn("HourlySalary",
                                 UDF.get_HourlySalaryUDF(
                                            col('Salary Range From'),
                                            col('Salary Range To'),
                                            col('Salary Frequency')
                                        )
                                     )


In [14]:
df_salary.limit(10).toPandas()

,Job ID,Agency,Job Category,Salary Range From,Salary Range To,Salary Frequency,Minimum Qual Requirements,Posting Date,HourlySalary
0,87990,DEPARTMENT OF BUSINESS SERV.,None,42405.00,65485.00,Annual,"""1.\tA baccalaureate degree from an accredited...",New York City residency is generally required ...,30.700001
1,97899,DEPARTMENT OF BUSINESS SERV.,None,60740.00,162014.00,Annual,"""1. A baccalaureate degree from an accredited ...",2012-01-26T00:00:00.000,63.389999
2,132292,NYC HOUSING AUTHORITY,Maintenance & Operations,51907.68,54580.32,Annual,"""1. Three years of full-time satisfactory expe...",None,30.299999
3,132292,NYC HOUSING AUTHORITY,Maintenance & Operations,51907.68,54580.32,Annual,"""1. Three years of full-time satisfactory expe...",None,30.299999
4,133921,NYC HOUSING AUTHORITY,Maintenance & Operations,35.00,35.00,Hourly,1. Five years of full-time satisfactory experi...,2014-01-09T00:00:00.000,35.000000
5,133921,NYC HOUSING AUTHORITY,Maintenance & Operations,35.00,35.00,Hourly,1. Five years of full-time satisfactory experi...,2014-01-09T00:00:00.000,35.000000
6,137433,DEPT OF HEALTH/MENTAL HYGIENE,"Finance, Accounting, & Procurement",50598.00,85053.00,Annual,"""1. A baccalaureate degree from an accredited ...",Apply online with a cover letter to https://a1...,38.599998
7,138531,DEPT OF ENVIRONMENT PROTECTION,"Health Public Safety, Inspections, & Enforcement",50623.00,75083.00,Annual,Qualification Requirements A baccalaureate de...,2013-12-20T00:00:00.000,35.770000
8,151131,NYC HOUSING AUTHORITY,"Engineering, Architecture, & Planning",90000.00,110000.00,Annual,"""1. A master's degree from an accredited colle...",None,56.919998
9,152738,LAW DEPARTMENT,Clerical & Administrative Support,30683.00,49707.00,Annual,Qualification Requirements A four-year high s...,2014-06-26T00:00:00.000,22.879999


# Minimum Qual Requirements

Required: a function to extrapolate any qualification specification. I may end up having a simply flag: required_degree: boolean

I have used [List of Degrees: Most Popular Degree Programs by Level](https://bestaccreditedcolleges.org/articles/list-of-degrees.html) as my reference

**PLEASE NOTE**: get_degreeUDF was tested [here](test%20UDF%20functions.ipynb#test_get_degreeUDF)

Call UDF get_degreeUDF(MinimumQualRequirements) to get the higher qualification requirements if found creating new column 
- "Higher Degree"

In [37]:
importlib.reload(UDF)
df_salary_degree = df_salary.withColumn("Higher Degree",
                                 UDF.get_degreeUDF(
                                            col('Minimum Qual Requirements')
                                        )
                                     )


# Job Category

- there are 2 missing Job Catergory's

For these I will set to "Not Specified"

In [78]:
from pyspark.sql.functions import col,regexp_extract

In [59]:
df_salary_degree.select('Job Category').distinct().count()

131

In [41]:
df_salary_degree.where(col('Job Category').isNull()).count()

2

In [40]:
df_salary_degree.where(col('Job Category').isNull()).show()

+------+--------------------+------------+-----------------+---------------+----------------+-------------------------+--------------------+------------+-------------+
|Job ID|              Agency|Job Category|Salary Range From|Salary Range To|Salary Frequency|Minimum Qual Requirements|        Posting Date|HourlySalary|Higher Degree|
+------+--------------------+------------+-----------------+---------------+----------------+-------------------------+--------------------+------------+-------------+
| 87990|DEPARTMENT OF BUS...|        null|          42405.0|        65485.0|          Annual|     "1.	A baccalaurea...|New York City res...|        30.7|       Degree|
| 97899|DEPARTMENT OF BUS...|        null|          60740.0|       162014.0|          Annual|     "1. A baccalaurea...|2012-01-26T00:00:...|       63.39|       Degree|
+------+--------------------+------------+-----------------+---------------+----------------+-------------------------+--------------------+------------+-------

In [50]:
from pyspark.sql.functions import when


In [64]:
df_salary_degree_JC = df_salary_degree.\
                        withColumn('Job Category', 
                                   when(
                                       col('Job Category').isNull(),"Not Specified").\
                                   otherwise(col('Job Category')))


rows with Job ID in (87990,97899) had NULL 'Job Category' before the update
check values now:

In [65]:
df_salary_degree_JC.where(col('Job Category').isNull()).count()

0

In [67]:
df_salary_degree_JC.filter(col('Job ID') == 87990).select('Job Category').show()

+-------------+
| Job Category|
+-------------+
|Not Specified|
+-------------+



In [68]:
df_salary_degree_JC.filter(col('Job ID') == 97899).select('Job Category').show()

+-------------+
| Job Category|
+-------------+
|Not Specified|
+-------------+



In [70]:
df_salary_degree_JC.filter(col('Job Category') == 'Not Specified').count()

2

In [62]:
df_salary_degree_JC.select('Job Category').distinct().count()

131

# Posting Date

See my analysis regarding [posting data](data_analysis.ipynb#posting_date_analysis_summary)

In [99]:
from pyspark.sql.functions import col,regexp_extract,to_date

In [ ]:
regexp_extract('str', r'(\d+)-(\d+)', 1)

In [75]:
df_salary_degree_JC.select('Posting Date').show(30,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Posting Date                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [121]:
df_pre_final = df_salary_degree_JC.\
            withColumn("PostingDateYYYYMMDD",
                      regexp_extract(
                          'Posting Date',
                          r'.*?(\d{4}-\d{2}-\d{2})T.*?',
                          1
                      )).\
            withColumn("PostingDate",
                       to_date(col("PostingDateYYYYMMDD"),"yyyy-MM-dd"))

In [122]:
df_pre_final.select("PostingDateYYYYMMDD","PostingDate").show()

+-------------------+-----------+
|PostingDateYYYYMMDD|PostingDate|
+-------------------+-----------+
|                   |       null|
|         2012-01-26| 2012-01-26|
|               null|       null|
|               null|       null|
|         2014-01-09| 2014-01-09|
|         2014-01-09| 2014-01-09|
|                   |       null|
|         2013-12-20| 2013-12-20|
|               null|       null|
|         2014-06-26| 2014-06-26|
|                   |       null|
|                   |       null|
|                   |       null|
|         2014-10-09| 2014-10-09|
|         2014-10-09| 2014-10-09|
|         2014-10-08| 2014-10-08|
|                   |       null|
|                   |       null|
|         2014-11-18| 2014-11-18|
|                   |       null|
+-------------------+-----------+
only showing top 20 rows



In [123]:
df_pre_final.printSchema()

root
 |-- Job ID: integer (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Job Category: string (nullable = true)
 |-- Salary Range From: double (nullable = true)
 |-- Salary Range To: double (nullable = true)
 |-- Salary Frequency: string (nullable = true)
 |-- Minimum Qual Requirements: string (nullable = true)
 |-- Posting Date: string (nullable = true)
 |-- HourlySalary: float (nullable = true)
 |-- Higher Degree: string (nullable = true)
 |-- PostingDateYYYYMMDD: string (nullable = true)
 |-- PostingDate: date (nullable = true)



In [126]:
df_final = df_pre_final.drop(col('Salary Range From')).\
                        drop(col('Salary Range To')).\
                        drop(col('Salary Frequency')).\
                        drop(col('Posting Date')).\
                        drop(col('Minimum Qual Requirements')).\
                        drop(col('PostingDateYYYYMMDD'))

In [127]:
df_final.printSchema()

root
 |-- Job ID: integer (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Job Category: string (nullable = true)
 |-- HourlySalary: float (nullable = true)
 |-- Higher Degree: string (nullable = true)
 |-- PostingDate: date (nullable = true)



In [ ]:
df_final.write.parquet('/processed/df_final',mode='overwrite')